To install JSONlines

In [50]:
!pip install jsonlines

Importing the dependencies

In [0]:
import tweepy
import json
import jsonlines
import pandas as pd

In [0]:
consumer_key = 'chWJ0x9OI21ptDccD6jWiQlxf'
consumer_secret = 'p2mWRlFMftyIspFqeRPhbK5eUhbjMLLte3jwk8Dl8mFv6vBPxA'

access_token = '1102633539734986752-5cMvmoIvR1C9RLT9vuS2JDUdJdtR2T'
access_token_secret = '2SZgWcSLvshXbwL5NfGeSYvIXLYUt0EPC5wQYzyTAE75r'

authorize twitter, initialize tweepy

In [0]:

# Authorization to consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# Access to user's access key and access secret
auth.set_access_token(access_token, access_token_secret)

# Calling api 
api = tweepy.API(auth)         

Function to fetch all the tweets made by a twitter handle

In [0]:
def get_tweets(username, api):
  
  alltweets = [] # a list to hold all the Tweets
  
  # request for most recent tweets (200 is the maximum allowed count)
  new_tweets = api.user_timeline(screen_name = username,count=199)
  
  alltweets.extend(new_tweets)
  
  # save the id of the oldest tweet less one
  # makes sure we keep fetching older tweets as we loop through rather than fetching same tweets 
  oldest = alltweets[-1].id - 1
  
  # Stay in the loop as long as new tweets are available to fetch  
  while len(new_tweets) > 0:
    
    new_tweets = api.user_timeline(screen_name = username,count=199,max_id=oldest)
    alltweets.extend(new_tweets)
    oldest = alltweets[-1].id - 1

    
    print("Total tweets downloaded from %s are %s" % (username,len(alltweets)))
  return alltweets
    

                             

Fetching twitter in a list

In [124]:
data = get_tweets('midasIIITD', api)
print(len(data))

Total tweets downloaded from midasIIITD are 344
Total tweets downloaded from midasIIITD are 344
344


Function to store all the tweets in JSON line file

In [0]:
def store_tweets(alltweets):
  
  tweet_list=[] # a list of all formatted tweets
  
  #looping through all the tweets in the list
  for tweet in alltweets:
    l = []
    tweet_information=dict() # a dict to contain information about single tweet
    
    # code to check for images present in tweet
    for image in  tweet.entities.get('media', []): # returns an empty list if no image is present
      l = image['media_url'] # return a list of image url
    if(len(l)>0):
      tweet_information['image_count'] = len(l)
    else:
      tweet_information['image_count'] = None
          
    # text of tweet                
    tweet_information['text']=tweet.text
    #date and time of tweet
    tweet_information['created_at']=tweet.created_at.strftime("%Y-%m-%d %H:%M:%S")
    
    # id of this tweet
    tweet_information['id_str']=tweet.id_str
    # retweet count
    tweet_information['retweet_count']=tweet.retweet_count
    
    #favourites count
    tweet_information['favorite_count']=tweet.favorite_count
    #username of the user to which it was replied (is Nullable)
    tweet_information['in_reply_to_screen_name']=tweet.in_reply_to_screen_name
    
    # user information in user dictionery
    user_dictionery=tweet._json['user']
    # no of followers of the user
    tweet_information['followers_count']=user_dictionery['followers_count']
    # screename of the person who tweeted this
    tweet_information['screen_name']=user_dictionery['screen_name']
    
    
    # add this tweet to the dictionary
    tweet_list.append(tweet_information)
    
  with jsonlines.open('/content/twitter.jsonl', mode='w') as writer:
    writer.write(tweet_list)


In [0]:
store_tweets(data)

To repesent the data from JSON line file in a tabular format using dataframe

In [0]:
with open("/content/twitter.jsonl") as datafile:
    data = json.load(datafile)
dataframe = pd.DataFrame(data)

In [128]:
dataframe.columns

Index(['created_at', 'favorite_count', 'followers_count', 'id_str',
       'image_count', 'in_reply_to_screen_name', 'retweet_count',
       'screen_name', 'text'],
      dtype='object')

Creating a new dataframe to display only the required columns

In [0]:
df = dataframe[['created_at', 'text', 'favorite_count', 'retweet_count', 'image_count']]

In [130]:
print(df.head())

            created_at                                               text  \
0  2019-04-10 09:01:29  Clarification: Our earlier post which indicate...   
1  2019-04-10 04:51:26  RT @IIITDelhi: Applications open for MTech (CB...   
2  2019-04-09 16:45:07  RT @IIITDelhi: We are delighted to share that ...   
3  2019-04-09 05:04:27  RT @Harvard: Professor Jelani Nelson founded A...   
4  2019-04-09 05:04:11  RT @emnlp2019: For anyone interested in submit...   

   favorite_count  retweet_count  image_count  
0               0              0          NaN  
1               0              1          NaN  
2               0             14          NaN  
3               0             35          NaN  
4               0             17          NaN  


To count the number of tweets which contain images

In [131]:
count = 0
for i in range(len(df)):
  if(df.iloc[i]['image_count']>0):
    count = count+1
print(count)

24
